In [1]:
import duckdb
import pandas as pd

In [2]:
# Change as needed
BASE_DIR = r'C:\Users\lukef\Documents\projects\fetch_rewards'
ASAT_DT = '2021-03-01' # Set to the latest month in the receipts data

In [3]:
data_dir = BASE_DIR + '/data'
raw_data_dir = data_dir + '/raw'
cleansed_data_dir = data_dir + '/cleansed'

In [4]:
dim_brands = cleansed_data_dir + '/dim_brands.parquet'
dim_users = cleansed_data_dir + '/dim_users.parquet'
fact_receipts = cleansed_data_dir + '/fact_receipts.parquet'
fact_receipt_items = cleansed_data_dir + '/fact_receipt_items.parquet'

In [5]:
# Questions 1 and 2
def top_brands_by_month(num_months_ago, num_brands = 5):
  """
  Retrieves the top brands by receipt count for a specified number of months ago,
  limited to a set number of brands, using DuckDB SQL queries.
  """
  sql_query = f"""
  SELECT
    b.brandCode,
    b.name AS brandName,
    COUNT(*) AS receiptCount
  FROM
    '{fact_receipt_items}' ri
      INNER JOIN
        '{fact_receipts}' r
        ON ri.receiptId = r.receiptId
      INNER JOIN
        '{dim_brands}' b
        ON ri.brandCode = b.brandCode
  WHERE
    DATE_TRUNC('MONTH', r.scannedDateTime) = DATE_TRUNC('MONTH', DATE '{ASAT_DT}' - INTERVAL '{num_months_ago} MONTH')
  GROUP BY
    b.brandCode,
    b.name
  ORDER BY
    receiptCount DESC
  LIMIT {num_brands}
  """

  return duckdb.query(sql_query).df()



In [6]:
# What are the top 5 brands by receipts scanned for most recent month?
top_brands_by_month(1)

,brandCode,brandName,receiptCount
0,VIVA,Viva,1


In [7]:
# How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
top_brands_by_month(2)

,brandCode,brandName,receiptCount
0,PEPSI,Pepsi,93
1,KLEENEX,Kleenex,88
2,KNORR,KNORR,79
3,DORITOS,Doritos,77
4,KRAFT,Kraft,60


In [9]:
# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
# When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
sql_query = f"""
WITH item_counts AS (
  SELECT
    receiptId,
    COUNT(DISTINCT receiptItemId) AS itemCount
  FROM
    '{fact_receipt_items}'
  GROUP BY
    receiptId
)
SELECT
  rewardsReceiptStatus,
  COUNT(DISTINCT r.receiptId) AS receiptCount,
  SUM(ic.itemCount) AS totalItemCount,
  AVG(totalSpent) AS averageSpent
FROM
  '{fact_receipts}' r
  LEFT JOIN
    item_counts ic
    ON r.receiptId = ic.receiptId
GROUP BY
  rewardsReceiptStatus
"""
duckdb.query(sql_query).df()

,rewardsReceiptStatus,receiptCount,totalItemCount,averageSpent
0,REJECTED,71,164.0,23.326056
1,SUBMITTED,434,NaN,NaN
2,FLAGGED,46,810.0,180.451739
3,FINISHED,518,5918.0,80.854305
4,PENDING,50,49.0,28.032449


In [10]:
# Questions about new users
sql_query = f"""
SELECT
  b.brandCode,
  b.name AS brandName,
  COUNT(*) AS receiptCount,
  SUM(totalSpent) AS totalSpent
FROM
  '{fact_receipt_items}' ri
    INNER JOIN
      '{fact_receipts}' r
      ON ri.receiptId = r.receiptId
    INNER JOIN
      '{dim_brands}' b
      ON ri.brandCode = b.brandCode
    INNER JOIN
      '{dim_users}' u
      ON r.userId = u.userId
WHERE
  u.createdDateTime >= DATE '{ASAT_DT}' - INTERVAL '6 MONTH'
GROUP BY
  b.brandCode,
  b.name
ORDER BY
  COUNT(*) DESC
"""
new_user_brands = duckdb.query(sql_query).df()

In [11]:
# Which brand has the most spend among users who were created within the past 6 months?
new_user_brands.sort_values(by='totalSpent', ascending=False).head(1)

,brandCode,brandName,receiptCount,totalSpent
0,PEPSI,Pepsi,118,119703.34


In [12]:
# Which brand has the most transactions among users who were created within the past 6 months?
new_user_brands.sort_values(by='receiptCount', ascending=False).head(1)

,brandCode,brandName,receiptCount,totalSpent
0,PEPSI,Pepsi,118,119703.34
